## 1. Import

In [1]:
import sys
import gc
import os
import re
import time
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
# import chromedriver_autoinstaller

from matplotlib import font_manager, rc
# 차트에서 한글 출력을 위한 설정

import matplotlib.pyplot as plt
import platform
your_os = platform.system()
if your_os == 'Linux':
    rc('font', family='NanumGothic')
elif your_os == 'Windows':
    ttf = "c:/Windows/Fonts/malgun.ttf"
    font_name = font_manager.FontProperties(fname=ttf).get_name()
    
    rc('font', family=font_name)
elif your_os == 'Darwin':
    rc('font', family='AppleGothic')
rc('axes', unicode_minus=False)

In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common. by import By

from bs4 import BeautifulSoup 

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import konlpy
from konlpy.tag import Okt
from konlp.kma.klt2023 import klt2023
import nltk
import stanza
from gensim.models import Word2Vec, word2vec
from gensim.models import FastText

from wordcloud import WordCloud
from numpy import dot
from numpy.linalg import norm
import itertools

## 2. Set Driver

In [131]:
gc.collect()

0

In [132]:
chrome_options = webdriver.ChromeOptions()

In [133]:
chrome_driver = './chromedriver.exe'
driver = webdriver.Chrome(chrome_driver)

In [134]:
driver.get("http://www.kyobobook.co.kr/")
time.sleep(3)

In [135]:
# 로그인
driver.find_element("link text", "로그인").click()
time.sleep(2)

## 본인 실제 교보문고 아이디, 비밀번호를 입력
ID = 'qwapol13'
PW = 'ogw!9517'

driver.find_element(By.XPATH, '//*[@id="mainDiv"]/main/section/div[1]/div[1]/div[1]/div/input').send_keys(ID)
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="mainDiv"]/main/section/div[1]/div[1]/div[2]/div/input').send_keys(PW)
time.sleep(2)

driver.find_element(By.XPATH, '//*[@id="mainDiv"]/main/section/div[1]/div[3]/button/span').click()
time.sleep(2)

In [136]:
# 목차 클릭
driver.find_element(By.XPATH, '//*[@id="welcome_header_wrap"]/div[3]/nav/div[1]/button').click()
time.sleep(3)

# 연간 베스트 확인하기 위해 이동
driver.find_element("link text", "국내도서 전체").click() 
time.sleep(3)

In [9]:
# 크롤링 결과값을 담을 list 생성
title_list = []
artist_list = []
ex_list= []
category_list = []

## 3. Crawling

제목, 저자, 설명, 카테고리, 베스트셀러 선정 기간 크롤링

In [122]:
for k in tqdm_notebook(range(21,23)):
    
    if (k==14) or (k==18):
        pass
    else:
        #카테고리 클릭
        cate = driver.find_element(By.XPATH, f'//*[@id="contents"]/div/aside/div[2]/div[1]/ul/li[{k}]/a')
        category = cate.text
        print(category)
        cate.click() 
        time.sleep(1)
        
        driver.find_element("link text", "베스트셀러").click() 
        time.sleep(5)

        
        for j in tqdm_notebook(range(1,13)):
            for i in range(1, 21):
                
                # 책 클릭
                try:
                    link = driver.find_element(By.XPATH, f'//*[@id="homeTabBest"]/div[3]/ol/li[{i}]/div[2]/div[2]/a')
                    link.send_keys(Keys.CONTROL +"\n") 
                    driver.switch_to.window(driver.window_handles[-1])
                except:
                    driver.find_element(By.XPATH, f'//*[@id="//*[@id="allTopPagi"]/div/a[{j-6}]').click()
                    time.sleep(5)
                    link = driver.find_element(By.XPATH, f'//*[@id="homeTabBest"]/div[3]/ol/li[{i}]/div[2]/div[2]/a')
                    link.send_keys(Keys.CONTROL +"\n") 
                    driver.switch_to.window(driver.window_handles[-1])
                    
                # 책 제목
                try:
                    title = driver.find_element(By.XPATH, '//*[@id="contents"]/div[1]/div/div[1]/div/div[1]/div[1]/div/h1/span')
                    title_list.append(title.text)
                except:
                    time.sleep(100+i)
                    driver.close()
                    driver.switch_to.window(driver.window_handles[-1])
                    time.sleep(3)
                    
                    link = driver.find_element(By.XPATH, f'//*[@id="homeTabBest"]/div[3]/ol/li[{i}]/div[2]/div[2]/a')
                    link.send_keys(Keys.CONTROL +"\n") 
                    driver.switch_to.window(driver.window_handles[-1])
                    
                    title = driver.find_element(By.XPATH, '//*[@id="contents"]/div[1]/div/div[1]/div/div[1]/div[1]/div/h1/span')
                    title_list.append(title.text)

                # 저자
                artist = driver.find_element(By.CSS_SELECTOR, 'div.prod_author_box.auto_overflow_wrap > div.auto_overflow_contents > div > div.author > a')
                artist_list.append(artist.text)

                # 설명
                try:
                    explain = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.book_intro > div.intro_bottom > div:nth-child(2)') 
                    ex_list.append(explain.text)
                except:
                    try:
                        explain = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.book_intro > div.intro_bottom > div:nth-child(1)')
                        ex_list.append(explain.text)
                    except:
                        try:
                            explain = driver.find_element(By.CSS_SELECTOR, '#scrollSpyProdInfo > div.product_detail_area.book_intro > div.intro_bottom > div')
                            ex_list.append(explain.text)
                        except:
                            ex_list.append(title.text)

                # 카테고리 
                category_list.append(category)

                driver.close()
                driver.switch_to.window(driver.window_handles[-1])
                time.sleep(2)

            # 다음 페이지 클릭
            if j==7:
                print(j,'page 완료!')
                driver.find_element(By.XPATH, '//*[@id="bestBottomPagi"]/button[2]').click()
                time.sleep(4)
            elif j>7:
                if j==13:
                    print(j,'page 완료!')
                    pass
                else:
                    print(j,'page 완료!')
                    num = j-5
                    driver.find_element(By.XPATH, f'//*[@id="bestBottomPagi"]/div/a[{num}]').click()
                    time.sleep(5)
                        
            else:
                print(j,'page 완료!')
                driver.find_element(By.XPATH, f'//*[@id="bestBottomPagi"]/div/a[{j+1}]').click()
                time.sleep(5)

        print(len(title_list), len(artist_list), len(ex_list), len(category_list))
        driver.find_element("link text", "국내도서").click() 
        time.sleep(5)
    
driver.close()

  0%|          | 0/2 [00:00<?, ?it/s]

잡지


  0%|          | 0/12 [00:00<?, ?it/s]

1 page 완료!
2 page 완료!
3 page 완료!
4 page 완료!
5 page 완료!
6 page 완료!
7 page 완료!
8 page 완료!
9 page 완료!
10 page 완료!
11 page 완료!
12 page 완료!
4560 4560 4560 4560
청소년


  0%|          | 0/12 [00:00<?, ?it/s]

1 page 완료!



KeyboardInterrupt



In [148]:
print(len(title_list), len(artist_list), len(ex_list), len(category_list))
df = pd.DataFrame({'제목':title_list, '저자': artist_list, '책소개' : ex_list, '카테고리' : category_list})
df.to_csv("book_all.csv",index=False, encoding="utf-8-sig")

4800 4800 4800 4800
